# GRAPE for preparing a cat state using Adam Tutorial

<img src="../../_static/notebook-images/cat_adam_tut.png" alt="cat state" width="800"/>

In [ ]:
# ruff: noqa
import os

os.sys.path.append("../../../..")

In [1]:
# ruff: noqa
import feedback_grape.grape as fg
import jax.numpy as jnp
from feedback_grape.utils.operators import *
from feedback_grape.utils.states import basis, coherent
from feedback_grape.utils.tensor import tensor

## Step 1: initialize parameters

In [2]:
T = 1  # microsecond
num_of_intervals = 100
N = 30  # dimension of hilbert space
alpha = 1.5
phi = jnp.pi

In [3]:
hconj = lambda a: jnp.swapaxes(a.conj(), -1, -2)

In [4]:
chi = 0.2385 * (2 * jnp.pi)
mu_qub = 4.0
mu_cav = 8.0

## Step 2: define target and start states

In [5]:
psi0 = tensor(basis(2), basis(N))
cat_target_state = coherent(N, alpha) + jnp.exp(-1j * phi) * coherent(
    N, -alpha
)
psi_target = tensor(basis(2), cat_target_state)

## Step 3: Build the Hamiltonian

In [6]:
# Using Jaynes-Cummings model for qubit + cavity
def build_grape_format_ham():
    """
    Build Hamiltonian for given (complex) e_qub and e_cav
    """

    a = tensor(identity(2), destroy(N))
    adag = tensor(identity(2), create(N))
    n_phot = adag @ a
    sigz = tensor(sigmaz(), identity(N))
    sigp = tensor(sigmap(), identity(N))
    one = tensor(identity(2), identity(N))

    H0 = +(chi / 2) * n_phot @ (sigz + one)
    H_ctrl_qub = mu_qub * sigp
    H_ctrl_qub_dag = hconj(H_ctrl_qub)
    H_ctrl_cav = mu_cav * adag
    H_ctrl_cav_dag = hconj(H_ctrl_cav)

    H_ctrl = [H_ctrl_qub, H_ctrl_qub_dag, H_ctrl_cav, H_ctrl_cav_dag]

    return H0, H_ctrl

## Step 4: run GRAPE

In [7]:
# ouputs final_fidelity of 0.6719432851176165 (uses adam (a bit faster))
H0, H_ctrl = build_grape_format_ham()
fg.optimize_pulse(
    H0,
    H_ctrl,
    psi0,
    psi_target,
    num_t_slots=num_of_intervals,
    total_evo_time=T,
    type="state",
)

Iteration 0, _fidelity: 0.0024724328206271197
Iteration 10, _fidelity: 0.15421883050180796
Iteration 20, _fidelity: 0.19085989541209086
Iteration 30, _fidelity: 0.2669406390763248
Iteration 40, _fidelity: 0.3635698833197192
Iteration 50, _fidelity: 0.44721528339538286
Iteration 60, _fidelity: 0.48538932644191096
Iteration 70, _fidelity: 0.49484735750695147
Iteration 80, _fidelity: 0.49610175214636143
Iteration 90, _fidelity: 0.49669032081913606
Iteration 100, _fidelity: 0.4972059974432342
Iteration 110, _fidelity: 0.49778477756692535
Iteration 120, _fidelity: 0.49851225011737216
Iteration 130, _fidelity: 0.49952313856950187
Iteration 140, _fidelity: 0.5010129093400919
Iteration 150, _fidelity: 0.503315221897554
Iteration 160, _fidelity: 0.5068947532071107
Iteration 170, _fidelity: 0.5119807016825444
Iteration 180, _fidelity: 0.518345861049036
Iteration 190, _fidelity: 0.5268657581278989
Iteration 200, _fidelity: 0.5390626580583942
Iteration 210, _fidelity: 0.5549753586243263
Iteration 

result(control_amplitudes=Array([[ 7.11356854e-01,  1.15476510e+00, -1.71563319e-01,
         1.74888337e-01],
       [ 1.03017230e+00,  1.55352961e+00, -8.36719102e-02,
         3.30969375e-01],
       [ 6.29928350e-01,  1.32107110e+00, -2.94509658e-01,
         7.80493026e-02],
       [ 8.70487944e-01,  1.67910161e+00, -3.62116867e-02,
         2.22793396e-01],
       [ 4.07990605e-01,  1.57133958e+00, -2.27538518e-01,
         6.37222453e-01],
       [ 6.21249557e-01,  1.26229354e+00,  4.38428842e-02,
         4.87597671e-01],
       [ 4.88940608e-01,  1.81996295e+00,  4.59759053e-02,
         3.04313227e-02],
       [ 5.04302426e-01,  1.33257335e+00, -1.17888196e-01,
         1.37583372e-01],
       [ 2.48785250e-01,  1.47479573e+00, -1.89317926e-01,
        -3.17838088e-02],
       [ 3.38611061e-01,  1.67388213e+00,  4.10517674e-01,
         3.54216187e-02],
       [ 4.14815604e-01,  1.54925648e+00,  4.44317161e-02,
        -1.30689299e-01],
       [ 2.75272184e-01,  1.40654837e+0